In [1]:
from pathlib import Path  
from collections import defaultdict

# Use this to read in a CCD image
from astropy.nddata import CCDData

# Use this to respresent the coordinates of the variable star
from astropy.coordinates import SkyCoord

# We will use table to read and write tables of data as CSV files (or as FITS)
from astropy.table import Table

# We will use this to get the APASS stars in the field of view of an image
from stellarphot.differential_photometry import find_apass_stars

import requests
import pandas
import json

In [2]:
# Star chart parameters
var_star_name = 'V0533 Her'

# File for saving comparison star magnitudes in
apass_mags_comp_stars = 'apass-V0533-Her.csv'

# You chould not need to change these....
fov = 40
limiting_magnitude = 16

In [3]:
payload = {'format':'json',
           'star': var_star_name,
           'fov': fov,
           'maglimit': limiting_magnitude
          }

r = requests.get('https://app.aavso.org/vsp/api/chart', params = payload)

print(r.url)

food = r.json()

https://app.aavso.org/vsp/api/chart/?format=json&star=V0533+Her&fov=40&maglimit=16


In [4]:
var_coord = SkyCoord(ra=food['ra'], dec=food['dec'], unit=('hour', 'degree'))

In [5]:
if not food['photometry']:
    raise RuntimeError('No comparison stars for this variable -- request them from the AAVSO')


In [6]:
json = json.dumps(food['photometry'])
panda = pandas.read_json(json, orient = 'records')

In [7]:
panda

,auid,ra,dec,label,bands,comments
0,000-BCS-520,18:14:34.02,41:51:18.5,119,"[{'band': 'V', 'mag': 11.935, 'error': 0.096},...",
1,000-BCS-524,18:14:37.93,41:54:53.9,123,"[{'band': 'V', 'mag': 12.256, 'error': 0.066},...",
2,000-BCS-531,18:14:52.94,41:53:00.3,128,"[{'band': 'V', 'mag': 12.756, 'error': 0.074},...",
3,000-BCS-529,18:14:48.07,41:49:26.0,131,"[{'band': 'V', 'mag': 13.09, 'error': 0.005}, ...",
4,000-BCS-512,18:14:20.20,41:49:18.3,133,"[{'band': 'V', 'mag': 13.273, 'error': 0.006},...",
5,000-BCS-532,18:14:54.61,41:54:03.3,134,"[{'band': 'V', 'mag': 13.437, 'error': 0.064},...",
6,000-BCS-514,18:14:23.19,41:54:06.0,135,"[{'band': 'V', 'mag': 13.491, 'error': 0.0}, {...",
7,000-BCS-517,18:14:30.10,41:50:39.5,139,"[{'band': 'V', 'mag': 13.939, 'error': 0.005},...",
8,000-BCS-515,18:14:23.35,41:53:14.3,151,"[{'band': 'V', 'mag': 15.078, 'error': 0.005},...",
9,000-BCS-510,18:14:12.81,41:51:05.1,153,"[{'band': 'V', 'mag': 15.32, 'error': 0.013000...",


In [8]:
filepath = Path('aavso.csv') 

In [9]:
panda.to_csv(filepath) 

In [10]:
aavso_comps = Table.from_pandas(panda)

In [11]:
aavso_comps['coords'] = SkyCoord(ra=aavso_comps['ra'], dec=aavso_comps['dec'], unit=('hour', 'degree'))

In [12]:
# ccd = CCDData.read(sample_image)

In [13]:
apass, apass_low_error = find_apass_stars(var_coord, max_mag_error=0.1, max_color_error=0.2)

apass['coords'] = SkyCoord(ra=apass['RAJ2000'], dec=apass['DEJ2000'], unit=('degree', 'degree'))



## Match the comp stars to APASS stars

One of the columns of the APASS table will be called `coords` and will be a list of coordinate objects. Astropy coordinates have the magical ability to match themselves to the coordinates of another set of coordinats.

You could use either the full apass list or the low error one below.

In [14]:
index, d2d, _ = aavso_comps['coords'].match_to_catalog_sky(apass['coords'])

In [15]:
if (d2d.arcsec > 1.5).any():
    raise RuntimeError('Coordinate mismatch')

In [16]:
apass_comp_stars = apass[index]

In [17]:
apass_comp_stars['auid'] = aavso_comps['auid']

In [18]:
apass_mag_cols_band_map = dict(
    Vmag='V', Bmag='B', g_mag='SG', r_mag='SR', i_mag='SI'
)

mag_err_cols = {k: f'e_{k}' for k in apass_mag_cols_band_map.keys()}

In [19]:
del apass_comp_stars['coords'], apass_comp_stars['B-V'], apass_comp_stars['e_B-V']

In [20]:
constant_cols = set(apass_comp_stars.colnames) - set(apass_mag_cols_band_map.keys()) - set(mag_err_cols.values())

In [21]:
rearranged = defaultdict(list)
n_bands = len(apass_mag_cols_band_map.keys())

for row in apass_comp_stars:
    for cc in constant_cols:
        rearranged[cc].extend([row[cc]] * n_bands)
    for band, band_name in apass_mag_cols_band_map.items():
        rearranged['band'].append(band_name)
        rearranged['mag'].append(row[band])
        rearranged['mag_err'].append(row[mag_err_cols[band]])

In [22]:
rearranged = Table(rearranged)

In [23]:
rearranged

e_RAJ2000,RAJ2000,Field,auid,DEJ2000,e_DEJ2000,mobs,nobs,recno,band,mag,mag_err
float32,float64,int64,str11,float64,float32,int16,int16,int32,str2,float32,float32
0.373,273.641704,20121948,000-BCS-520,41.855157,0.246,36,4,14537579,V,11.861,0.042
0.373,273.641704,20121948,000-BCS-520,41.855157,0.246,36,4,14537579,B,12.277,0.116
0.373,273.641704,20121948,000-BCS-520,41.855157,0.246,36,4,14537579,SG,12.039,0.053
0.373,273.641704,20121948,000-BCS-520,41.855157,0.246,36,4,14537579,SR,11.759,0.052
0.373,273.641704,20121948,000-BCS-520,41.855157,0.246,36,4,14537579,SI,11.677,0.055
0.469,273.657989,20121948,000-BCS-524,41.914956,0.493,36,4,14537585,V,12.174,0.066
0.469,273.657989,20121948,000-BCS-524,41.914956,0.493,36,4,14537585,B,13.28,0.114
0.469,273.657989,20121948,000-BCS-524,41.914956,0.493,36,4,14537585,SG,12.708,0.059
0.469,273.657989,20121948,000-BCS-524,41.914956,0.493,36,4,14537585,SR,11.81,0.045


In [24]:
rearranged.write(apass_mags_comp_stars, overwrite=True)